In [ ]:
import langchain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

In [ ]:
import os
with open('../OPENAI_API_KEY', 'r') as file:
    env_value = file.read().strip()
os.environ['OPENAI_API_KEY'] = env_value

In [ ]:
os.environ['OPENAI_API_KEY'] = env_value

In [ ]:
model = ChatOpenAI(model_name="gpt-3.5-turbo")

In [ ]:
#model.invoke("Explain concept of derivatives in mathematics for kid?")

In [ ]:
template = (
    """You are playing the role of an educator. You teach {grade_level} students about {course_subject}. 

    This is how I want you to interact with my students:
    {ai_policy}
    """
)
bot_message_prompt = SystemMessagePromptTemplate.from_template(template)
student_template = "{text}"
student_message_prompt = HumanMessagePromptTemplate.from_template(student_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [bot_message_prompt, student_message_prompt]
)

# get a chat completion from the formatted messages
model.invoke(
    chat_prompt.format_prompt(
        grade_level="first year undergraduates", 
        course_subject="introductory biology", 
        ai_policy= "I want the students to learn critical thinking. Under no circumstances are you supposed to give the students the full answers to questions on the first try. Try and help the student get to the question on their own three times before giving the answer.",
        text="What is the world's largest animal?"
    ).to_messages()
)

In [ ]:
model.invoke?


In [ ]:
embeddings = OpenAIEmbeddings()
output_parser = StrOutputParser()
filepath = "STP 420 spring 2024 course syllabus.pdf" #file as input
pages = "" #extracting file content here
def load_pdf(filepath):
    path = filepath
    loader = PyPDFLoader(path) 
    pages = loader.load_and_split() #loading file content as document with page content
    db = FAISS.from_documents(pages, embeddings)
    return db
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
# docs = text_splitter.split_documents(pages)

In [ ]:
db =  load_pdf(filepath)
db.index.ntotal

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

def load_pdf(filepath):
    loader = UnstructuredMarkdownLoader(filepath, mode="elements") #mode=elements breaks up the text into chunks
    doc = loader.load()
    db = FAISS.from_documents(doc, embeddings)
    return db

In [ ]:
db2 = load_pdf("2017-01-29-policies.md")
db2.index.ntotal

In [ ]:
db.merge_from(db2)
db.index.ntotal

In [ ]:
def get_response_from_query(db, query, k = 4):
    docs = db.similarity_search(query, k=2)
    docs_page_content = " ".join([d.page_content for d in docs])
    template="You are the world renowned problem solver and useful teacher who can answer questions based on course content: {docs}"
    prompt = ChatPromptTemplate.from_messages([("system",template), ("user", query)])
    chain = prompt | llm | output_parser
    return chain.invoke({"input": query, "docs": docs_page_content, "query": query})
response = get_response_from_query(db, "when will we learn confidence interval?")
print(response)